In [1]:
!pip install -q timm

In [12]:
from urllib.request import urlopen
from PIL import Image
import timm

img = Image.open(
    urlopen(
        "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/beignets-task-guide.png"
    )
)

model = timm.create_model(
    "convnext_xxlarge.clip_laion2b_soup_ft_in1k",
    pretrained=True,
    num_classes=0,  # remove classifier nn.Linear
)
model = model.eval()

# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

output = model(
    transforms(img).unsqueeze(0)
)  # output is (batch_size, num_features) shaped tensor

# or equivalently (without needing to set num_classes=0)

# output = model.forward_features(transforms(img).unsqueeze(0))
# output is unpooled, a (1, 3072, 8, 8) shaped tensor

# output = model.forward_head(output, pre_logits=True)
# output is a (1, num_features) shaped tensor

In [13]:
transforms

Compose(
    Resize(size=256, interpolation=bicubic, max_size=None, antialias=warn)
    CenterCrop(size=(256, 256))
    ToTensor()
    Normalize(mean=tensor([0.4815, 0.4578, 0.4082]), std=tensor([0.2686, 0.2613, 0.2758]))
)

In [14]:
timm.data.create_transform(**data_config, is_training=True)

Compose(
    RandomResizedCropAndInterpolation(size=(256, 256), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bicubic)
    RandomHorizontalFlip(p=0.5)
    ColorJitter(brightness=(0.6, 1.4), contrast=(0.6, 1.4), saturation=(0.6, 1.4), hue=None)
    ToTensor()
    Normalize(mean=tensor([0.4815, 0.4578, 0.4082]), std=tensor([0.2686, 0.2613, 0.2758]))
)

In [2]:
output.shape

torch.Size([1, 3072])

In [7]:
!python preprocess_timm.py

device:  cuda
data_dir:  data/train
total_data_points:  2944
File data/timm_preprocessed_train.npy does not exist - creating a new one ...
saved file: data/timm_preprocessed_train.npy
data_dir:  data/validation
total_data_points:  1466
File data/timm_preprocessed_validation.npy does not exist - creating a new one ...
saved file: data/timm_preprocessed_validation.npy
data_dir:  data/test
total_data_points:  1463
File data/timm_preprocessed_test.npy does not exist - creating a new one ...
saved file: data/timm_preprocessed_test.npy


In [1]:
from train_v6 import *

data, labels = load_training_data()

# Define the dataset and dataloader
dataset = JigsawDataset(data, labels)
print(f"dataset len: {len(dataset)}")

dataset, dataset[0][0].shape

loaded training data from: data/timm_preprocessed_train.npy
dataset len: 2944


(<train_v6.JigsawDataset at 0x7efc998d8d50>, torch.Size([36, 3072]))

In [4]:
import torch
from thop import profile
from thop import clever_format
from JigsawNet import JigsawNet

model = JigsawNet(n_classes=50, num_features=3072)

x = dataset[0][0]
flops, params = profile(model, inputs=(x,))
flops, params = clever_format([flops, params], "%.3f")

print(flops, params)

[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm1d'>.
425.927M 370.898M


In [9]:
%%time

!python train_v6.py -e 10 -b 64 2>&1 | tee logs/v6-e-10-b-64.txt

device: cuda
epochs:  10 batch 64
loaded training data from: data/timm_preprocessed_train.npy
dataset len: 2944
train_set len: 2356
val_set len: 588
Epoch 1, Learning rate: 0.0010000000
Epoch 1, Training Loss: 3.9123, Training Accuracy: 1.87%
Epoch 1, Validation Loss: 3.9176, Validation Accuracy: 2.55%
saved checkpoint: /home/inflaton/code/projects/cs712/data/checkpoints/checkpoint-000.pth
Epoch 2, Learning rate: 0.0010000000
Epoch 2, Training Loss: 3.9109, Training Accuracy: 5.26%
Epoch 2, Validation Loss: 3.9223, Validation Accuracy: 2.55%
saved checkpoint: /home/inflaton/code/projects/cs712/data/checkpoints/checkpoint-001.pth
Epoch 3, Learning rate: 0.0010000000
Epoch 3, Training Loss: 3.9088, Training Accuracy: 6.75%
Epoch 3, Validation Loss: 3.9191, Validation Accuracy: 1.70%
saved checkpoint: /home/inflaton/code/projects/cs712/data/checkpoints/checkpoint-002.pth
Epoch 4, Learning rate: 0.0010000000
Epoch 4, Training Loss: 3.8891, Training Accuracy: 14.26%
Epoch 4, Validation Loss

In [10]:
%%time

!python train_v6.py -e 100 -b 64 2>&1 | tee logs/v6-e-100-b-64.txt

device: cuda
epochs:  100 batch 64
loaded training data from: data/timm_preprocessed_train.npy
dataset len: 2944
train_set len: 2356
val_set len: 588
Epoch 1, Learning rate: 0.0010000000
Epoch 1, Training Loss: 3.9123, Training Accuracy: 1.87%
Epoch 1, Validation Loss: 3.9176, Validation Accuracy: 2.55%
saved checkpoint: /home/inflaton/code/projects/cs712/data/checkpoints/checkpoint-000.pth
Epoch 2, Learning rate: 0.0010000000
Epoch 2, Training Loss: 3.9109, Training Accuracy: 5.26%
Epoch 2, Validation Loss: 3.9223, Validation Accuracy: 2.55%
saved checkpoint: /home/inflaton/code/projects/cs712/data/checkpoints/checkpoint-001.pth
Epoch 3, Learning rate: 0.0010000000
Epoch 3, Training Loss: 3.9088, Training Accuracy: 6.75%
Epoch 3, Validation Loss: 3.9191, Validation Accuracy: 1.70%
saved checkpoint: /home/inflaton/code/projects/cs712/data/checkpoints/checkpoint-002.pth
Epoch 4, Learning rate: 0.0010000000
Epoch 4, Training Loss: 3.8891, Training Accuracy: 14.26%
Epoch 4, Validation Los

In [11]:
%%time

!python train_v6.py -e 100 -b 64 2>&1 | tee logs/v6-e-100-b-64-step-25.txt

device: cuda
epochs:  100 batch 64
loaded training data from: data/timm_preprocessed_train.npy
dataset len: 2944
train_set len: 2356
val_set len: 588
Epoch 1, Learning rate: 0.0010000000
Epoch 1, Training Loss: 3.9123, Training Accuracy: 1.87%
Epoch 1, Validation Loss: 3.9176, Validation Accuracy: 2.55%
saved checkpoint: /home/inflaton/code/projects/cs712/data/checkpoints/checkpoint-000.pth
Epoch 2, Learning rate: 0.0010000000
Epoch 2, Training Loss: 3.9109, Training Accuracy: 5.26%
Epoch 2, Validation Loss: 3.9223, Validation Accuracy: 2.55%
saved checkpoint: /home/inflaton/code/projects/cs712/data/checkpoints/checkpoint-001.pth
Epoch 3, Learning rate: 0.0010000000
Epoch 3, Training Loss: 3.9088, Training Accuracy: 6.75%
Epoch 3, Validation Loss: 3.9191, Validation Accuracy: 1.70%
saved checkpoint: /home/inflaton/code/projects/cs712/data/checkpoints/checkpoint-002.pth
Epoch 4, Learning rate: 0.0010000000
Epoch 4, Training Loss: 3.8891, Training Accuracy: 14.26%
Epoch 4, Validation Los

In [9]:
%%time

!python train_v7.py -e 20 -b 64 2>&1 | tee logs/v7-e-20-b-64.txt

device: cuda
epochs:  20 batch 64
loaded training data from: data/preprocessed_train_0.npy
loaded training data from: data/preprocessed_train_1.npy
loaded training data from: data/preprocessed_train_2.npy
loaded training data from: data/preprocessed_train_3.npy
loaded training data from: data/preprocessed_train_4.npy
loaded training data from: data/preprocessed_train_5.npy
loaded training data from: data/preprocessed_train_6.npy
loaded training data from: data/preprocessed_train_7.npy
loaded training data from: data/preprocessed_train_8.npy
loaded training data from: data/preprocessed_train_9.npy
dataset len: 29440
data_points: 2944
train_set len: 23560
val_set len: 5880
Epoch 1, Learning rate: 0.0010000000
Epoch 1, Training Loss: 3.7902, Training Accuracy: 27.87%
Epoch 1, Validation Loss: 3.8315, Validation Accuracy: 23.49%
saved checkpoint: /home/inflaton/code/projects/cs712/data/checkpoints/checkpoint-000.pth
Epoch 2, Learning rate: 0.0010000000
Epoch 2, Training Loss: 3.2504, Train

In [11]:
%%time

!python train_v7.py -e 10 -b 64 2>&1 | tee logs/v7-e-10-b-64.txt

device: cuda
epochs:  10 batch 64
loaded training data from: data/preprocessed_train_0.npy
loaded training data from: data/preprocessed_train_1.npy
loaded training data from: data/preprocessed_train_2.npy
loaded training data from: data/preprocessed_train_3.npy
loaded training data from: data/preprocessed_train_4.npy
loaded training data from: data/preprocessed_train_5.npy
loaded training data from: data/preprocessed_train_6.npy
loaded training data from: data/preprocessed_train_7.npy
loaded training data from: data/preprocessed_train_8.npy
loaded training data from: data/preprocessed_train_9.npy
dataset len: 29440
data_points: 2944
train_set len: 26500
val_set len: 2940
Epoch 1, Learning rate: 0.0010000000
Epoch 1, Training Loss: 3.7601, Training Accuracy: 28.72%
Epoch 1, Validation Loss: 3.7441, Validation Accuracy: 29.39%
saved checkpoint: /home/inflaton/code/projects/cs712/data/checkpoints/checkpoint-000.pth
Epoch 2, Learning rate: 0.0010000000
Epoch 2, Training Loss: 3.2659, Train

In [13]:
%%time

!python preprocess_timm_v2.py

device:  cuda
data_dir:  data/train
total_data_points:  2944
File data/timm_preprocessed_train_0.npy does not exist - creating a new one ...
saved file: data/timm_preprocessed_train_0.npy
File data/timm_preprocessed_train_1.npy does not exist - creating a new one ...
saved file: data/timm_preprocessed_train_1.npy
File data/timm_preprocessed_train_2.npy does not exist - creating a new one ...
saved file: data/timm_preprocessed_train_2.npy
File data/timm_preprocessed_train_3.npy does not exist - creating a new one ...
saved file: data/timm_preprocessed_train_3.npy
File data/timm_preprocessed_train_4.npy does not exist - creating a new one ...
saved file: data/timm_preprocessed_train_4.npy
File data/timm_preprocessed_train_5.npy does not exist - creating a new one ...
saved file: data/timm_preprocessed_train_5.npy
File data/timm_preprocessed_train_6.npy does not exist - creating a new one ...
saved file: data/timm_preprocessed_train_6.npy
File data/timm_preprocessed_train_7.npy does not e